In [2]:
!pip install sentencepiece
!pip install protobuf


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 2.1 MB/s eta 0:00:001.4 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from transformers import pipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentiment_pipeline = pipeline("sentiment-analysis", device=device,
                              model='cardiffnlp/twitter-xlm-roberta-base-sentiment')

In [16]:
import pandas as pd
df=pd.read_csv('dataset/train.csv', encoding='unicode_escape')
df.drop(['textID', 'selected_text', 'Time of Tweet', 'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)'], axis=1, inplace=True)
#mask = df['sentiment'] == 'neutral'
#df=df[~mask]
#mask= df['text'].str.len() < 100
#df=df[~mask]
df=df[:1000]
print (f'The dataset has {len(df)} lines!')
df.head(10)

The dataset has 1000 lines!


,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
5,http://www.dothebouncy.com/smf - some shameles...,neutral
6,2am feedings for the baby are fun when he is a...,positive
7,Soooo high,neutral
8,Both of you,neutral
9,Journey!? Wow... u just became cooler. hehe....,positive


In [17]:
text_data=df['text'].to_numpy().astype(str).tolist()
sentiment=df['sentiment'].to_numpy().astype(str).tolist()
print(text_data[:10])

[' I`d have responded, if I were going', ' Sooo SAD I will miss you here in San Diego!!!', 'my boss is bullying me...', ' what interview! leave me alone', ' Sons of ****, why couldn`t they put them on the releases we already bought', 'http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth', '2am feedings for the baby are fun when he is all smiles and coos', 'Soooo high', ' Both of you', ' Journey!? Wow... u just became cooler.  hehe... (is that possible!?)']


In [18]:
%time output=sentiment_pipeline(text_data)

CPU times: user 5.6 s, sys: 2.17 ms, total: 5.61 s
Wall time: 5.6 s


In [19]:
print(output[:10])

[{'label': 'neutral', 'score': 0.5402872562408447}, {'label': 'negative', 'score': 0.9343284368515015}, {'label': 'negative', 'score': 0.945700466632843}, {'label': 'negative', 'score': 0.8988997340202332}, {'label': 'negative', 'score': 0.8394114375114441}, {'label': 'negative', 'score': 0.543731689453125}, {'label': 'positive', 'score': 0.8367201089859009}, {'label': 'negative', 'score': 0.4926181435585022}, {'label': 'neutral', 'score': 0.4310312271118164}, {'label': 'positive', 'score': 0.8752656579017639}]


In [20]:
predict=[item['label'].lower() for item in output]
print ('Predicted:', predict[:10])
print ('Real:     ', sentiment[:10])

Predicted: ['neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'neutral', 'positive']
Real:      ['neutral', 'negative', 'negative', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'positive']


In [21]:
count_good=0
count_bad=0
length=len(sentiment)
for i in range(0, length):
    if predict[i]==sentiment[i]:
        count_good+=1
    else:
        count_bad+=1
accuracy=count_good*100/length
loss=count_bad*100/length
print(f'This model has an accuracy of {accuracy}%!')
print(f'This model has a loss of {loss}%!')

This model has an accuracy of 69.8%!
This model has a loss of 30.2%!


**Raport:** 
1. Accuracy of 83% when classifying between Positive and Negative only.
2. Accuracy of 69% when trying to match everything!